# Logging feedback of queries to ES

## Creating index

In [ ]:
import os
os.environ['TF_CACHE_DIR'   ] = '/var/tmp/models'

ES_USERNAME = 'elastic'
ES_PASSWORD = 'changeme'
ES_FEEDBACK_INDEX = 'feedback'

## select the environment for ingestion
# ES_HOST = 'http://localhost:9200/'
ES_HOST = 'https://dev.es.chat.ask.eduworks.com/'
# ES_HOST = 'https://qa.es.chat.ask.eduworks.com/'

In [ ]:
pipeline_id = 'transform_id'
mapping  = {
    "pipeline": {
        "id"    : pipeline_id,
        "body"  : {
            "description"   : "Replace the _id with chat_id for the logs index",
            "processors"    : [{
                "set": {
                    "field": "_id",
                    "value": "{{chat_id}}"
                }
            }]
        }
    },
    "settings": {
        "number_of_shards"  : 2, 
        "number_of_replicas": 1,
        "default_pipeline"  : pipeline_id
    },
    "mappings": {
        "dynamic"   : "false",
        "_source"   : {"enabled": "true"},
        "properties": {
            "chat_id"   : {"type": "keyword", "index": "true", "doc_values": "false", "ignore_above": 256},
            "timestamp" : {"type": "date"   , "index": "true", "doc_values": "true"},
            "query"     : {"type": "keyword", "index": "true", "doc_values": "false", "ignore_above": 256},
            "feedback"  : {"type": "keyword", "index": "true", "doc_values": "false", "ignore_above": 256},
            "setting"   : {
                "dynamic"       : "true",
                "type"          : "nested",
            },
        }
    }
}

In [ ]:
from elasticsearch import Elasticsearch, RequestError
from elasticsearch.client import IngestClient


# increase the timeout if necessary
es_client = Elasticsearch([ES_HOST], http_auth=(ES_USERNAME, ES_PASSWORD), timeout = 20)
es_ingest = IngestClient(es_client)

# create pipeline for replacing _id with chat_id
es_ingest.put_pipeline(
    id   = mapping['pipeline']['id'     ],
    body = mapping['pipeline']['body'   ])

# create index
es_client.indices.delete(
    index   = ES_FEEDBACK_INDEX, 
    ignore  = 404)
es_client.indices.create(
    index       = ES_FEEDBACK_INDEX   , 
    settings    = mapping['settings']       , 
    mappings    = mapping['mappings']       )

es_client.indices.refresh()

## Reading sample events json file and transforming logs

In [ ]:
from datetime import datetime
import uuid

feedback = {
    'chat_id'   : "some_id" + '_' + uuid.uuid4().hex[:4],
    'timestamp' : datetime.now().isoformat(),
    'query'     : 'Yellow pests all over the kitchen',
    'feedback'  : 'yes',
    'settings'  : {
        'client'        : 'ucipm',
        'es_cut_off'    : 0.4,
        'es_downweight' : 1
    }
}

In [ ]:
try:
    response = es_client.index(
        index       = ES_FEEDBACK_INDEX     ,
        document    = feedback              ,
        id          = feedback['chat_id']   ,
    )
except RequestError as e:
    raise(e)

try:
    assert response['result'] in ['created', 'updated']
except AssertionError as e:
    raise e

response

## Retrieving feedback

In [ ]:
# date format = `dd.mm.yyyy`
def _parse_date(aft_date = None, bfr_date = None):

    
    try:
        if aft_date is None:
            aft_date = datetime.min
        else:

            aft_date = datetime.strptime(aft_date, '%d.%m.%Y')
        
        if bfr_date is None:
            bfr_date = datetime.max
        else:
            bfr_date = datetime.strptime(bfr_date, '%d.%m.%Y')

        aft_date = aft_date.isoformat()
        bfr_date = bfr_date.isoformat()

    except (TypeError, ValueError) as e:
        print(f'Input(s) should be string in the format `dd.mm.yyyy`')
        raise(e)

    return aft_date, bfr_date


# aft_date = None
# bfr_date = None
aft_date = '05.05.2021'
bfr_date = None
aft_date, bfr_date = _parse_date(
    aft_date = aft_date, 
    bfr_date = bfr_date
)    
    

query = {
    "range": {"timestamp": {
        'gte': aft_date,
        'lte': bfr_date 
    }}
}

response = es_client.search(
    index   = ES_FEEDBACK_INDEX,
    query   = query                     
)

response